In [26]:
# STEP 1: Install required packages
!pip install easyocr googletrans==4.0.0-rc1 opencv-python-headless ffmpeg-python

# STEP 2: Import libraries
import easyocr
from googletrans import Translator
import cv2
import numpy as np
from PIL import Image
import tempfile
import os
from google.colab import files
from IPython.display import display
import ipywidgets as widgets

# STEP 3: Initialize OCR and Translator
reader = easyocr.Reader(['en'])
translator = Translator()

# STEP 4: Language selection
language_options = {
    'hi': 'Hindi', 'fr': 'French', 'es': 'Spanish', 'ta': 'Tamil', 'te': 'Telugu',
    'de': 'German', 'zh-cn': 'Chinese (Simplified)', 'ja': 'Japanese', 'ar': 'Arabic',
    'ru': 'Russian', 'it': 'Italian', 'ml': 'Malayalam', 'kn': 'Kannada',
    'bn': 'Bengali', 'pa': 'Punjabi'
}

lang_select = widgets.SelectMultiple(
    options=[(name, code) for code, name in language_options.items()],
    value=[],
    description='Languages:',
    disabled=False
)

print("Please select at least one target language for translation:")
display(lang_select)

def get_selected_languages():
    return list(lang_select.value)

Please select at least one target language for translation:


SelectMultiple(description='Languages:', options=(('Hindi', 'hi'), ('French', 'fr'), ('Spanish', 'es'), ('Tami…

In [27]:
# STEP 5: Frame extraction from video
def extract_frames(video_path, max_frames=5):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        count += 1
    cap.release()
    return frames

# STEP 6: Text extraction and translation
def process_file(file, target_langs):
    extracted_english_words = []

    # Consider expanding this tuple to include other common image formats like .png, .jpeg, etc.
    image_extensions = ('/content/Quote.mp4') # Added common image extensions and corrected format
    if file.name.lower().endswith(image_extensions):
        # Ensure the file-like object is positioned at the beginning before reading
        file.seek(0)
        img = Image.open(file)
        img_np = np.array(img.convert("RGB")) # Ensure image is in RGB format
        results = reader.readtext(img_np)
    else:  # Assume it's a video if it's not a recognized image extension
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp:
            # Ensure the file-like object is positioned at the beginning before reading
            # This is necessary because the file-like object might have been read before
            file.seek(0)
            tmp.write(file.read())
            tmp_path = tmp.name

        frames = extract_frames(tmp_path)
        results = []
        for frame in frames:
            results += reader.readtext(frame)

        os.remove(tmp_path)

    for detection in results:
        text = detection[1]
        # Added a check for empty strings or strings with only whitespace
        if text.strip() and all(c.isalpha() or c.isspace() for c in text.strip()):
            extracted_english_words.append(text.strip())

    combined_text = ' '.join(extracted_english_words).strip()
    if not combined_text:
        return "No English text found.", {}

    translations = {}
    # Ensure target_langs is a list before iterating
    if isinstance(target_langs, list):
        for lang in target_langs:
            try:
                translated = translator.translate(combined_text, dest=lang)
                translations[lang] = translated.text
            except Exception as e:
                translations[lang] = f"Translation error: {str(e)}"
    else: # Handle the case where target_langs is not a list (e.g., None or empty)
         print("Warning: No target languages provided for translation.")
         # Optionally return an error message or handle differently
         # translations['error'] = "No target languages specified."


    return combined_text, translations

# STEP 7: Enforce language selection before proceeding
selected_languages = get_selected_languages()
if not selected_languages:
    print("You must select at least one language before uploading a file.")
else:
    print("Please upload an image or video file (e.g., .jpg, .png, .mp4).")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Exiting.")
    else:
        file_name = next(iter(uploaded))

        class FileMock:
            # Corrected the spelling of the constructor
            def __init__(self, file_path):
                # Open the actual file in binary read mode
                self.file = open(file_path, "rb")
                self.name = file_path
            def read(self):
                # Read the entire content of the file
                return self.file.read()
            def close(self):
                # Close the underlying file
                self.file.close()
            def seek(self, offset, whence=0):
                # Implement the seek method by calling seek on the underlying file
                return self.file.seek(offset, whence)

        # Create an instance of FileMock with the uploaded file name
        mock_file = FileMock(file_name)

        # Process the file using the defined function
        extracted, translations = process_file(mock_file, selected_languages) # Pass selected_languages

        # Close the mock file object (which closes the actual file)
        mock_file.close()

        print("\nExtracted English Text:\n", extracted)
        print("\nTranslated Texts:")
        for lang, translated_text in translations.items():
            lang_name = language_options.get(lang, lang)
            print(f"{lang_name} ({lang}): {translated_text}")

Please upload an image or video file (e.g., .jpg, .png, .mp4).


Saving Quote.mp4 to Quote (2).mp4

Extracted English Text:
 KINEMASTER Old friends ae New friends are dont forget gold because only can hold a diamond gold KINEMASTER Old friends ae New friends are dont forget gold because only can hold a diamond gold KINEMASTER Old friends ae New friends are dont forget gold because only can hold a diamond gold KINEMASTER Old friends ae New friends are dont forget gold because only can hold a diamond gold KINEMASTER Old friends ae New friends are dont forget gold because only can hold a diamond gold

Translated Texts:
Hindi (hi): किनेमास्टर पुराने दोस्त ऐ नए दोस्त सोने को नहीं भूल रहे हैं क्योंकि केवल एक हीरे के सोने की किनमास्टर पुराने दोस्त एई नए दोस्त सोने को नहीं भूल सकते हैं क्योंकि केवल एक हीरे के सोने की किनेमास्टर पुराने दोस्त एई नए दोस्त नहीं हैं।
